# Hampus EDA 2

In [2]:
import pandas as pd
import plotly.express as px
import matplotlib
import duckdb as db
import plotly.graph_objects as go
from utils.constants import DATA_DIRECTORY

In [3]:
df_kurser = pd.read_excel(
    DATA_DIRECTORY / "resultat-2024-for-kurser-inom-yh.xlsx", sheet_name="Lista ansökningar"
).drop(
    columns=[
        "Diarienummer",
        "Kommun",
        "Län",
        "FA-region",
        "Antal kommuner",
        "Antal län",
        "Antal FA-regioner",
    ])


In [4]:
def approved_courses_filter(df):
    approved_courses = db.query(
        """--sql
        SELECT
            "Anordnare namn" AS anordnare,
            COUNT(*) FILTER (WHERE Beslut ILIKE '%Bevilj%') AS "antal beviljade",
            COUNT(*) FILTER (WHERE Beslut ILIKE '%Avslag%') AS "antal avslag",
        FROM df
        GROUP BY "Anordnare namn"
        ORDER BY "antal beviljade" DESC;
        """
    ).df()
    return approved_courses

In [5]:
filtered_df = approved_courses_filter(df_kurser)

In [6]:
def plot_bar(df):
    df_long = df.head(5).melt(
    id_vars="anordnare",
    value_vars=["antal beviljade", "antal avslag"],
    var_name="Beslut",
    value_name="Antal",
)

    # Create the bar chart
    fig = px.bar(
        df_long,
        x="Antal",
        y="anordnare",
        color="Beslut",
        orientation="h",
        title="Antal beviljade/avslag ",
    )

    fig.update_layout(
        yaxis={"categoryorder": "total ascending"},
        xaxis_title="Antal",
        yaxis_title="Anordnare",
        legend_title="Beslut",
        bargap=0.2,
    )

    return fig

In [11]:
def plot_bar(df, antal=10, highlight=None):
    resultat = df.sort_values("antal beviljade", ascending=False).head(antal)

    fig = go.Figure()

    beviljade_colors = []
    avslag_colors = []

    for område in resultat["anordnare"]:
        if highlight is None or highlight == "Visa alla" or område == highlight:
            beviljade_colors.append("rgba(74, 140, 232, 1)") 
            avslag_colors.append("rgba(211, 211, 211, 1)")
        else:
            beviljade_colors.append("rgba(74, 140, 232, 0.3)") 
            avslag_colors.append("rgba(211, 211, 211, 0.3)")

    fig.add_trace(go.Bar(
        y=resultat['anordnare'],
        x=resultat['antal beviljade'],
        name='antal beviljade',
        orientation='h',
        marker=dict(color=beviljade_colors),
        hovertemplate="<b>%{y}</b><br> Beviljade <br> Antal beslut: %{x} <br><extra></extra>"
    ))

    fig.add_trace(go.Bar(
        y=resultat['anordnare'],
        x=resultat['antal avslag'],
        name='antal avslag',
        orientation='h',
        marker=dict(color=avslag_colors),
        hovertemplate="<b>%{y}</b><br> Avslag <br> Antal beslut: %{x} <br><extra></extra>"
    ))

    fig.update_layout(
        barmode='stack',
        title='Beviljade och avslagna kurser per område (2024)',
        xaxis_title='Antal',
        yaxis_title='Skola',
        template='simple_white',
        showlegend=False,
        yaxis=dict(autorange="reversed"),
        title_font=dict(family="Times New Roman", size=30, weight="bold"),
        font=dict(family="Times New Roman", size=15),
        yaxis_tickfont=dict(family="Times New Roman", size=15, weight="bold"),
        annotations=[dict(
            x=-0.7,
            y=-0.2,
            xref="paper",
            yref="paper",
            text="<b>Data source:</b> Myndigheten för yrkeshögskolan",
            showarrow=False,
        )]
    )

    return fig

In [12]:
plot_bar(filtered_df.head(), highlight="YH Akademin AB")